In [47]:
import pandas as pd

In [48]:
df_train=pd.read_csv('/Users/chidam/Desktop/kaggle_tweet sentiment extraction/tweet_sentiment_train.csv')

In [49]:
len(df_train)

27481

In [50]:
df_train.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [51]:
# import json
import tensorflow as tf
import csv
import random
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers


In [52]:
oov_tok = "<OOV>"
max_len=141#looped through all the sentences and figured the max length

In [53]:
training_portion=0.99
train_size = int(len(df_train['text'])*training_portion)



In [8]:
train_sentences = df_train['text'][:train_size]
train_labels = df_train['selected_text'][:train_size]

validation_sentences = df_train['text'][train_size:]
validation_labels = df_train['selected_text'][train_size:]



In [42]:
train_labels

0                      I`d have responded, if I were going
1                                                 Sooo SAD
2                                              bullying me
3                                           leave me alone
4                                            Sons of ****,
                               ...                        
27201                           is gaining weight.  rather
27202                   Believe it or not I`m a super geek
27203                                  Screw all vacuums!!
27204    I have too much work to drink now.   but I do ...
27205                            missing my car SO MUCH!!!
Name: selected_text, Length: 27206, dtype: object

In [43]:
validation_labels

27206                i should be finished.... but i`m not.
27207                                           yucky burn
27208                                                 miss
27209                                             es faile
27210                                 Soooo humid outside!
                               ...                        
27476                                               d lost
27477                                        , don`t force
27478                            Yay good for both of you.
27479                           But it was worth it  ****.
27480    All this flirting going on - The ATG smiles. Y...
Name: selected_text, Length: 275, dtype: object

In [9]:
len(train_sentences)

27206

In [10]:
len(validation_sentences)

275

In [14]:
train_sentences

0                      I`d have responded, if I were going
1            Sooo SAD I will miss you here in San Diego!!!
2                                my boss is bullying me...
3                           what interview! leave me alone
4         Sons of ****, why couldn`t they put them on t...
                               ...                        
27201                           is gaining weight.  rather
27202                   Believe it or not I`m a super geek
27203    I just vacuumed my entire house three times. S...
27204     I have too much work to drink now.   but I do...
27205    Left Cali this morning...in Dallas now..missin...
Name: text, Length: 27206, dtype: object

In [18]:
type(train_sentences[0])

str

In [44]:
def find_maxlength(l):

    max_len=0
    for i in l:
        
        max_len=max(max_len, len(str(i)))
        
    return max_len
    
    

In [22]:
find_maxlength(train_sentences)

141

In [45]:
find_maxlength(df_train['selected_text'])

141

In [ ]:
train_sentences = df_train['text'][:train_size]
train_labels = df_train['selected_text'][:train_size]

validation_sentences = df_train['text'][train_size:]
validation_labels = df_train['selected_text'][train_size:]


In [29]:
#changing type of every sentence into string:

def change_type_sentence(x, y, c, z):
    train=[]
    train_lab=[]
    validation=[]
    validation_lab=[]
    
    for i in x:
        train.append(str(i))
        
    for j in y:
        train_lab.append(str(i))
        
    for i in c:
        validation.append(str(i))
        
    for i in z:
        validation_lab.append(str(i))
        
    return train, train_lab, validation, validation_lab
        
        



In [30]:
train, train_lab, validation, validation_lab=change_type_sentence(train_sentences, train_labels, validation_sentences, validation_labels)



In [31]:
len(train)

27206

In [32]:
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(train)
word_index = tokenizer.word_index
print(len(word_index))

26447


In [36]:
max_len=141
vocab_size = 25000
embedding_dim = 16
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

In [35]:
train_sequences = tokenizer.texts_to_sequences(train)
train_padded = pad_sequences(train_sequences, maxlen=max_len)

validation_sequences = tokenizer.texts_to_sequences(validation)
validation_padded = pad_sequences(validation_sequences,maxlen=max_len)

In [37]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_len),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32)),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 141, 16)           400000    
_________________________________________________________________
bidirectional (Bidirectional (None, 64)                9600      
_________________________________________________________________
dense (Dense)                (None, 6)                 390       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 409,997
Trainable params: 409,997
Non-trainable params: 0
_________________________________________________________________


In [46]:
num_epochs = 5
# history = model.fit(train_padded, train_lab, epochs=num_epochs, validation_data=(validation_padded, validation_lab))

history = model.fit(np.array(train_padded), np.array(train_lab), epochs=num_epochs, validation_data=(np.array(validation_padded), np.array(validation_lab)), verbose=2)



Train on 27206 samples, validate on 275 samples
Epoch 1/5


UnimplementedError:  Cast string to float is not supported
	 [[node loss/dense_1_loss/Cast (defined at /usr/local/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1751) ]] [Op:__inference_distributed_function_6118]

Function call stack:
distributed_function
